#### Introduction
*Every news website classifies the news article before publishing it so that every time visitors visit their website can easily click on the type of news that interests them. Currently, the news articles are classified by hand by the content managers of news websites. But to save time, they can also implement a machine learning model on their websites that read the news headline or the content of the news and classifies the category of the news.*

#### Text Classification
**